In [26]:
from __future__ import print_function
import pandas as pd
import os
import logging
from peewee import *
from playhouse.sqlite_ext import SqliteExtDatabase
import requests
import w3lib.url
import json
import copy

In [2]:
df = pd.read_csv("input.csv")
post_code_list = df["post_code"]

In [4]:
request_url = "https://zoeken.kvk.nl/JsonSearch.ashx?q=1100AC&prefproduct=&prefpayment=&start=40"

In [5]:
post_code_list[0]

'1000AA'

In [11]:
url = w3lib.url.add_or_replace_parameter(request_url, "q", post_code_list[0])

In [13]:
response = requests.get(url)

In [19]:
json_obj = json.loads(response.text)

In [24]:
total_counts = int(int(json_obj['pageinfo']['resultscount'])/10)+1

In [30]:
print("post code `%s` has `%s` first pages" % (str(post_code_list[0]), str(total_counts)))

post code `1000AA` has `5` first pages


In [27]:
urls_of_post_code = []
for i in range(0, total_counts):
    each_url = w3lib.url.add_or_replace_parameter(request_url, "q", post_code_list[0])
    final_url = w3lib.url.add_or_replace_parameter(each_url, "start", str(i*10))
    urls_of_post_code.append(copy.deepcopy(final_url))

In [28]:
urls_of_post_code

['https://zoeken.kvk.nl/JsonSearch.ashx?q=1000AA&prefproduct=&prefpayment=&start=0',
 'https://zoeken.kvk.nl/JsonSearch.ashx?q=1000AA&prefproduct=&prefpayment=&start=10',
 'https://zoeken.kvk.nl/JsonSearch.ashx?q=1000AA&prefproduct=&prefpayment=&start=20',
 'https://zoeken.kvk.nl/JsonSearch.ashx?q=1000AA&prefproduct=&prefpayment=&start=30',
 'https://zoeken.kvk.nl/JsonSearch.ashx?q=1000AA&prefproduct=&prefpayment=&start=40']

In [35]:
class BaseModel(Model):
    class Meta:
        db = SqliteDatabase("temp_kvk_1.db")
        database = db.connect()

class kvkFirstStageModel(BaseModel):
    url = CharField(unique=True)
    post_code = CharField()

db = SqliteDatabase("temp_kvk_1.db")
db.connect()
try:
    db.drop_tables([kvkFirstStageModel])
except:
    pass
db.create_tables([kvkFirstStageModel])

In [41]:
for url_with_post_code in urls_of_post_code:
    item = kvkFirstStageModel()
    item.url = url_with_post_code
    item.post_code = post_code_list[0]
    try:
        item.save()
    except Exception as e:
        logging.info(e)